## Dataset
In this homework, we'll build a model for classifying various hair types. For this, we will use the Hair Type dataset that was obtained from [Kaggle](https://www.kaggle.com/datasets/kavyasreeb/hair-type-dataset) and slightly rebuilt.

You can download the target dataset for this homework from [here](https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip):
```bash
wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
unzip data.zip
```
In the lectures we saw how to use a pre-trained neural network. In the homework, we'll train a much smaller model from scratch.

>Note: you will need an environment with a GPU for this homework. We recommend to use [Saturn Cloud](https://bit.ly/saturn-mlzoomcamp). You can also use a computer without a GPU (e.g. your laptop), but it will be slower.

## Data Preparation
The dataset contains around 1000 images of hairs in the separate folders for training and test sets.

## Reproducibility
Reproducibility in deep learning is a multifaceted challenge that requires attention to both software and hardware details. In some cases, we can't guarantee exactly the same results during the same experiment runs. Therefore, in this homework we suggest to:

* install tensorflow version 2.17.1
* set the seed generators by:
```python
import numpy as np
import tensorflow as tf

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
```

In [1]:
# !wget https://github.com/SVizor42/ML_Zoomcamp/releases/download/straight-curly-data/data.zip
# !unzip data.zip

In [2]:
!pip install tensorflow==2.17.1

In [3]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

2024-12-01 12:04:50.005371: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-01 12:04:50.023252: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-12-01 12:04:50.042923: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-12-01 12:04:50.048818: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-01 12:04:50.062752: I tensorflow/core/platform/cpu_feature_guar

In [4]:
tf.__version__

'2.17.1'

In [5]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

train_dir = './data/train'
test_dir = './data/test'

## Model
For this homework we will use Convolutional Neural Network (CNN). Like in the lectures, we'll use Keras.

You need to develop the model with following structure:

* The shape for input should be `(200, 200, 3)`.
* Next, create a convolutional layer ([Conv2D](https://keras.io/api/layers/convolution_layers/convolution2d/)):
    * Use 32 filters.
    * Kernel size should be `(3, 3)` (that's the size of the filter).
    * Use `'relu'` as activation.
* Reduce the size of the feature map with max pooling ([MaxPooling2D](https://keras.io/api/layers/pooling_layers/max_pooling2d/)).
* Set the pooling size to `(2, 2)`.
* Turn the multi-dimensional result into vectors using a [Flatten](https://keras.io/api/layers/reshaping_layers/flatten/) layer
* Next, add a [Dense](https://keras.io/api/layers/core_layers/dense/) layer with 64 neurons and `'relu'` activation.
* Finally, create the `Dense` layer with 1 neuron - this will be the output.
    * The output layer should have an activation - use the appropriate activation for the binary classification case
As optimizer use [SGD](https://keras.io/api/optimizers/sgd/) with the following parameters:
    * `SGD(lr=0.002, momentum=0.8)`
For clarification about kernel size and max pooling, check [Office Hours](https://www.youtube.com/watch?v=1WRgdBTUaAc).

In [6]:
inputs = tf.keras.Input(shape=(200,200,3))
Conv2d = tf.keras.layers.Conv2D(filters=32, 
                                kernel_size=(3,3),
                                activation='relu')(inputs)
pooling = tf.keras.layers.MaxPooling2D(pool_size=(2,2))(Conv2d)
flatten = tf.keras.layers.Flatten()(pooling)
dense = tf.keras.layers.Dense(64, activation='relu')(flatten)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

I0000 00:00:1733054691.947270    4609 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733054691.994642    4609 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733054691.994985    4609 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
I0000 00:00:1733054691.996458    4609 cuda_executor.cc:1015] successful NUMA node read from SysFS ha

In [7]:
lr = 0.002
momentum = 0.8
optimizer = tf.keras.optimizers.SGD(learning_rate=lr, momentum=momentum)

loss = tf.keras.losses.BinaryCrossentropy(from_logits=False)

model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

## Question 1
Since we have a binary classification problem, what is the best loss function for us?

* `mean squared error`
* `binary crossentropy`
* `categorical crossentropy`
* `cosine similarity`

Note: since we specify an activation for the output layer, we don't need to set `from_logits=True`

**Answer**: binary crossentropy

## Question 2
What's the total number of parameters of the model? You can use the `summary` method for that.

* 896
* 11214912
* 15896912
* 20072512

In [8]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 200, 200, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 198, 198, 32)   │           896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 99, 99, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 313632)         │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │    20,072,512 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,073,473 (76.57 MB)

 Trainable params: 20,073,473 (76.57 MB)

 Non-trainable params: 0 (0.00 B)

## Generators and Training
For the next two questions, use the following data generator for both train and test sets:

[`ImageDataGenerator(rescale=1./255)`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator)
* We don't need to do any additional pre-processing for the images.
* When reading the data from train/test directories, check the `class_mode` parameter. Which value should it be for a binary classification problem?
* Use `batch_size=20`
* Use `shuffle=True` for both training and test sets.

For training use [.fit()](https://keras.io/api/models/model_training_apis/#fit-method) with the following params:
```python
model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)
```

In [9]:
train_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(train_dir,
                                                                    class_mode='binary',
                                                                    target_size=(200,200),
                                                                    batch_size=20,
                                                                    shuffle=True
                                                                    )

Found 800 images belonging to 2 classes.


In [10]:
test_generator = ImageDataGenerator(rescale=1./255).flow_from_directory(test_dir,
                                                                    class_mode='binary',
                                                                    target_size=(200,200),
                                                                    batch_size=20,
                                                                    shuffle=True
                                                                    )

Found 201 images belonging to 2 classes.


In [11]:
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=test_generator
)

/opt/saturncloud/envs/saturn/lib/python3.12/site-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10


I0000 00:00:1733054694.127678    4720 service.cc:146] XLA service 0x7f85700198b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1733054694.127717    4720 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
2024-12-01 12:04:54.146372: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-12-01 12:04:54.244431: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907


 4/40 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.5562 - loss: 0.7185

I0000 00:00:1733054695.537168    4720 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


40/40 ━━━━━━━━━━━━━━━━━━━━ 8s 153ms/step - accuracy: 0.5504 - loss: 0.7057 - val_accuracy: 0.6318 - val_loss: 0.6439
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 127ms/step - accuracy: 0.7151 - loss: 0.5808 - val_accuracy: 0.5622 - val_loss: 0.7156
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 117ms/step - accuracy: 0.6582 - loss: 0.6264 - val_accuracy: 0.6269 - val_loss: 0.6228
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 124ms/step - accuracy: 0.6945 - loss: 0.5591 - val_accuracy: 0.6318 - val_loss: 0.6328
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 121ms/step - accuracy: 0.7319 - loss: 0.5410 - val_accuracy: 0.6567 - val_loss: 0.5995
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 125ms/step - accuracy: 0.7371 - loss: 0.5298 - val_accuracy: 0.6517 - val_loss: 0.6139
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 124ms/step - accuracy: 0.7348 - loss: 0.5300 - val_accuracy: 0.6667 - val_loss: 0.6291
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 5s 126ms/step - accuracy: 0.7702 - loss: 0.4815 - val_accuracy: 0.6716 - val_

In [12]:
history.history.keys()

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])

## Question 3
What is the median of training accuracy for all the epochs for this model?

* 0.10
* 0.32
* 0.50
* 0.72

In [13]:
np.median(history.history['accuracy'])

0.7237499952316284

## Question 4
What is the standard deviation of training loss for all the epochs for this model?

* 0.028
* 0.068
* 0.128
* 0.168

In [14]:
np.std(history.history['loss'])

0.06696299591301712

## Data Augmentation
For the next two questions, we'll generate more data using data augmentations. See [ImageDataGenerator](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator).

Add the following augmentations to your training data generator:

* `rotation_range=50`,
* `width_shift_range=0.1`,
* `height_shift_range=0.1`,
* `zoom_range=0.1`,
* `horizontal_flip=True`,
* `fill_mode='nearest'`

In [15]:
data_augmentation = ImageDataGenerator(rescale=1./255,
                            rotation_range=50,
                            width_shift_range=0.1,
                            height_shift_range=0.1,
                            zoom_range=0.1,
                            horizontal_flip=True,
                            fill_mode='nearest'
                            )

In [16]:
train_generator_augment = data_augmentation.flow_from_directory(train_dir,
                                                class_mode='binary',
                                                target_size=(200,200),
                                                batch_size=20,
                                                shuffle=True
                                                )

Found 800 images belonging to 2 classes.


## Question 5
Let's train our model for 10 more epochs using the same code as previously.

>Note: make sure you don't re-create the model - we want to continue training the model we already started training.

What is the mean of test loss for all the epochs for the model trained with augmentations?

* 0.26
* 0.56
* 0.86
* 1.16

In [17]:
history_aug = model.fit(
    train_generator_augment,
    epochs=10,
    validation_data=test_generator
)

Epoch 1/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 268ms/step - accuracy: 0.6307 - loss: 0.6556 - val_accuracy: 0.6965 - val_loss: 0.5877
Epoch 2/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 10s 261ms/step - accuracy: 0.6814 - loss: 0.6069 - val_accuracy: 0.6716 - val_loss: 0.5659
Epoch 3/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 263ms/step - accuracy: 0.6642 - loss: 0.6125 - val_accuracy: 0.6915 - val_loss: 0.6010
Epoch 4/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 264ms/step - accuracy: 0.6855 - loss: 0.5898 - val_accuracy: 0.6965 - val_loss: 0.5653
Epoch 5/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 261ms/step - accuracy: 0.6817 - loss: 0.6261 - val_accuracy: 0.7114 - val_loss: 0.5774
Epoch 6/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 260ms/step - accuracy: 0.6879 - loss: 0.5827 - val_accuracy: 0.6368 - val_loss: 0.7231
Epoch 7/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 261ms/step - accuracy: 0.6788 - loss: 0.5691 - val_accuracy: 0.7065 - val_loss: 0.6050
Epoch 8/10
40/40 ━━━━━━━━━━━━━━━━━━━━ 11s 265ms/step - accuracy: 0.6701 - loss: 0.5926 - val_accu

In [18]:
print(len(history_aug.history['val_loss']))
np.mean(history_aug.history['val_loss'])

10


0.5904381155967713

## Question 6
What's the average of test accuracy for the last 5 epochs (from 6 to 10) for the model trained with augmentations?

* 0.31
* 0.51
* 0.71
* 0.91

In [19]:
np.mean(history_aug.history['val_accuracy'][5:])

0.6975124478340149